In [407]:
from common import *

import gzip
import os
import shutil
import sqlite3
from urllib.parse import urlparse

import duckdb
import pandas as pd
import requests
from oaklib import get_adapter
from oaklib.datamodels.vocabulary import IS_A
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import yaml
import json


In [408]:
print("verify output is being rendered")

verify output is being rendered


In [409]:
# Initialize cache dictionaries for predict_from_normalized_env_packages
# todo how to move the definitions for function that use these globals? Or just use caching around the function?
ancestor_cache = {}
descendant_cache = {}

In [410]:
# todo deal with circularity in env package prediction -> env triad reporting

# todo this on-demand NCBI curie extraction and annotation recapitulates work that is being added to
# https://portal.nersc.gov/project/m3408/biosamples_duckdb/
# via 
#   although that doesn't detect auto-incremented curies from  spreadsheet dragging

# todo if more caching is desired, it should probably take the form of saving dataframes for TSV

# eventually, dig up a complete JSON gold biosample dump for non-hybrid biosample counts

# overall run time (if NCBI biosamples and goldData are cached): ~ 10 minutes

# count studies not biosamples ?
# how? for gold, ncbi or nmdc?

# Task Settings
_For making a Soil env_broad_scale voting sheet vs a Sediment env_local_scale sheet, etc._

todo: bundle these into dicts so they don't have to be modified independently and kept in sync with one another.

In [411]:
output_file_name = "voting_sheets_output/plant_associated_env_local_scale_voting_sheet.tsv"

In [412]:
# semantic_anchor = 'ENVO:00000428' # biome for env_broad_scale
semantic_anchor = 'ENVO:01000813' # astronomical body part "abp" for env_local_scale
# semantic_anchor = 'ENVO:00010483' # environmental material for env_medium

In [413]:
plant_first_where = "s1.value like 'host-associated > plants%'"
sediment_first_where = "lower(s1.value) like 'environmental > aquatic%sediment%'"
soil_first_where = "s1.value like 'environmental > terrestrial > soil%'"
water_first_where = "s1.value like 'environmental > aquatic%' and lower(s1.value) not like '%sediment%'"

## context selectors

In [414]:
# gold_context_selector = 'mixs:env_broad'
gold_context_selector = 'mixs:env_local'
# gold_context_selector = 'mixs:env_medium'

In [415]:
# ncbi_context_selector = 'env_broad_scale'
ncbi_context_selector = 'env_local_scale'
# ncbi_context_selector = 'env_medium'

In [416]:
# nmdc_context_selector= 'env_broad_scale_id'
nmdc_context_selector= 'env_local_scale_id'
# nmdc_context_selector= 'env_medium_id'

## package aka environment aka extension selectors

In [417]:
gold_first_where = plant_first_where

In [418]:
# todo new since soil: why are we only considering MIMS.me for discovering appropriate env triad values?
#   there's usually a roughly equal number of biosamples from in each extension for MIMS.me and 

ncbi_package_selector = 'plant-associated.6.0'
# ncbi_package_selector = 'sediment.6.0'
# ncbi_package_selector = 'soil.6.0'
# ncbi_package_selector = 'water.6.0'

In [419]:
nmdc_package_selector = 'plant-associated'
# nmdc_package_selector = 'sediment'
# nmdc_package_selector = 'soil'
# nmdc_package_selector = 'water'


In [420]:
GOLDTERMS_NA = '' # ???

GOLDTERMS_PLANT_ASSOCIATED = GOLDTERMS_NA # host associated -> viridiplantae? take a string approach!
GOLDTERMS_SEDIMENT = 'GOLDTERMS:3985' #  doesn't have any subclasses
GOLDTERMS_SOIL = 'GOLDTERMS:4212'
GOLDTERMS_WATER = 'GOLDTERMS:3984'

# GOLDTERMS:4180, 'Environmental > Aquatic > Freshwater > Pond > Sediment' and ~64 more don't share a common root
# poetry run runoak -i sqlite:obo:goldterms info 't~sediment'


In [421]:
goldterms_root = GOLDTERMS_NA

## selecting name and version of one enum for comparison


In [422]:
# only the Soil enums have legacy definitions (v10.7 and earlier?)

# CONTEXT_ENUM = "EnvBroadScaleSoilEnum"
CONTEXT_ENUM = "EnvLocalScaleSoilEnum"
# CONTEXT_ENUM = "EnvMediumSoilEnum"

# CONTEXT_ENUM = ""

In [423]:
# todo: add columns for membership in multiple enums from multiple version of the schema?
#  like sediment local vs soil local and water local (once that's completed)
#  get them from schema files or something prior to that? sems like the voting sheets are too raw/preliminary for that
#   can use a more recent schema url for more recent enums!

# previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/v10.7.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

previous_submission_schema_url = "https://raw.githubusercontent.com/microbiomedata/submission-schema/refs/tags/v11.1.0/src/nmdc_submission_schema/schema/nmdc_submission_schema.yaml"

In [424]:
# todo: don't call the column "legacy_pv". use the name of the enum and the version of the schema?

# comparison_enum_column_name = 'EnvMediumSoilEnum_10_7'
comparison_enum_column_name = 'EnvLocalScaleSoilEnum_11_1'
# comparison_enum_column_name = 'no_comparison_enum'

# Additional Settings

In [425]:
# Approved prefixes (case-insensitive)
approved_prefixes = ['ENVO']

In [426]:
MIN_ANNOTATION_LEN = 3

In [427]:
NMDC_RUNTIME_BASE_URL = 'https://api.microbiomedata.org/nmdcschema/'
STUDY_SET_COLLECTION = 'study_set'
BIOSAMPLE_SET_COLLECTION = 'biosample_set'

In [428]:
envo_adapter_string = "sqlite:obo:envo"

In [429]:
# goldterms_adapter_string = "sqlite:obo:envo"

In [430]:
env_package_override_file = 'mam-env-package-overrides.tsv'
override_column = 'mam_inferred_env_package'

In [431]:
# ncbi_duckdb_url = 'https://portal.nersc.gov/project/m3408/biosamples_duckdb/ncbi_biosamples_2024-09-23.duckdb.gz'
ncbi_duckdb_url = 'https://portal.nersc.gov/project/m3408/biosamples_duckdb/ncbi_biosamples.duckdb.gz'

In [432]:
gold_data_url = "https://gold.jgi.doe.gov/download?mode=site_excel"
gold_data_file_name = "goldData.xlsx" # goldData.xlsx: Microsoft Excel 2007+
gold_csv_file_name = "gold_biosamples.csv"
BIOSAMPLES_SHEET = "Biosample"

In [433]:
goldterms_semsql_url = "https://s3.amazonaws.com/bbop-sqlite/goldterms.db.gz"


In [434]:
all_nmdc_biosamples_file = 'all_nmdc_biosamples.json'

# CURIe Constants

In [435]:
BIOME = 'ENVO:00000428'
TERRESTRIAL_BIOME = 'ENVO:00000446'
AQUATIC_BIOME = 'ENVO:00002030'
ABP = 'ENVO:01000813'
ENVIRONMENTAL_SYSTEM = 'ENVO:01000254'
ENVIRONMENTAL_MATERIAL = 'ENVO:00010483'

SOIL = 'ENVO:00001998'
LIQUID_WATER = 'ENVO:00002006'
WATER_ICE = 'ENVO:01000277'

HUMAN_CONSTRUCTION = 'ENVO:00000070'
BUILDING = 'ENVO:00000073'
BUILDING_PART = 'ENVO:01000420'

# Settings-based Queries

In [436]:
goldterms_subclass_query = f"""
select
	subject
from
	entailed_edge ee
where
	predicate = 'rdfs:subClassOf'
	and object = '{goldterms_root}'
"""

In [437]:
# todo could this have been done with a OAK query, eliminating the need to explicitly download the file?

goldterms_envo_query = f"""
SELECT
	*
FROM
	statements s
WHERE
	predicate = '{gold_context_selector}'"""

In [438]:
ncbi_query = f"""
SELECT content, COUNT(1) AS sample_count 
FROM attributes 
WHERE harmonized_name = '{ncbi_context_selector}' AND package_content like '%{ncbi_package_selector}'
GROUP BY content
ORDER BY COUNT(1) DESC
"""

In [439]:
# and s1.subject = s1.stanza eliminates matches on blank node anntoation rows (probably woudn't change results but adds a little overhead)

extension_query = f"""
select
		s1.subject ,
		s2.predicate,
		COALESCE (s2."object",
	s2."value") as content
from
	statements s1
join statements s2 on 
	s1.subject = s2.subject
where
	{gold_first_where}
	and s1.predicate = 'rdfs:label'
	and s1.subject = s1.stanza
	and s2.predicate in ('mixs:env_broad', 'mixs:env_local', 'mixs:env_medium', 'mixs:mixs_extension', 'rdfs:label', 'mixs:other', 'mixs:anatomical_site', 'mixs:host_taxon') ;
"""

# todo provide examples of sediment samples that are excluded by the requirement for aquatic

# Locally Defined Functions
_Currently using locally-defined cache dictionaries_

In [440]:
def predict_from_normalized_env_packages(df_raw, adapter):
    # Apply the function to the relevant columns

    df = df_raw.copy()

    print(df.shape)
    for column in ['env_broad_scale_id', 'env_local_scale_id', 'env_medium_id']:
        df[f'{column}_ancestors'] = df[column].apply(lambda x: get_hierarchy_terms(x, adapter)['ancestors'])
        df[f'{column}_descendants'] = df[column].apply(lambda x: get_hierarchy_terms(x, adapter)['descendants'])

    # Vectorize each set of terms separately
    broad_scale_ancestors = vectorize_terms(df, 'env_broad_scale_id_ancestors')
    broad_scale_descendants = vectorize_terms(df, 'env_broad_scale_id_descendants')

    local_scale_ancestors = vectorize_terms(df, 'env_local_scale_id_ancestors')
    local_scale_descendants = vectorize_terms(df, 'env_local_scale_id_descendants')

    medium_ancestors = vectorize_terms(df, 'env_medium_id_ancestors')
    medium_descendants = vectorize_terms(df, 'env_medium_id_descendants')

    # Combine all feature matrices
    X = hstack([
        broad_scale_ancestors,
        broad_scale_descendants,
        local_scale_ancestors,
        local_scale_descendants,
        medium_ancestors,
        medium_descendants
    ])

    # Filter the DataFrame to only include non-null rows for the target column
    df_filtered = df[df['normalized_env_package'].notnull() & (df['normalized_env_package'] != "")]

    # Extract the target variable
    y = df_filtered['normalized_env_package']

    # Ensure X corresponds to the filtered rows
    X_filtered = X[df_filtered.index]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_filtered, y, test_size=0.3, random_state=42)

    # Train a Random Forest Classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = clf.predict(X_test)

    # Evaluate the model
    print(classification_report(y_test, y_pred))

    # not determining confidence for each class nor saving any diagnostics any more

    return clf.predict(X)

In [441]:
def get_hierarchy_terms(my_curie: str, adapter) -> dict:
    """
    Extract ancestor and descendant terms from the ontology for a given CURIE,
    using caching to improve performance and filtering by 'is_a' relationships.

    Args:
        my_curie (str): CURIE identifier for the ontology term.
        adapter: Ontology adapter.

    Returns:
        dict: Dictionary containing lists of ancestor and descendant terms.
    """
    if my_curie not in ancestor_cache:
        try:
            ancestors = list(adapter.ancestors(my_curie, predicates=[IS_A]))
            ancestor_cache[my_curie] = [adapter.label(ancestor) for ancestor in ancestors if ancestor]
        except Exception as my_e:
            print(f"Error retrieving ancestors for {my_curie}: {my_e}")
            ancestor_cache[my_curie] = []

    if my_curie not in descendant_cache:
        try:
            descendants = list(adapter.descendants(my_curie, predicates=[IS_A]))
            descendant_cache[my_curie] = [adapter.label(descendant) for descendant in descendants if descendant]
        except Exception as my_e:
            print(f"Error retrieving descendants for {my_curie}: {my_e}")
            descendant_cache[my_curie] = []

    return {
        'ancestors': ancestor_cache[my_curie],
        'descendants': descendant_cache[my_curie],
    }

# Procedural Code Starts Here

In [442]:
# Determine the filenames and target directory for the NCBI DuckDB
ncbi_compressed_filename = urlparse(ncbi_duckdb_url).path.split('/')[-1]
ncbi_filename = os.path.splitext(ncbi_compressed_filename)[0]
ncbi_compressed_file_path = os.path.join(ncbi_compressed_filename)
ncbi_uncompressed_file_path = os.path.join(ncbi_filename)

# target_dir = os.path.join('.') # just assume the files are downloaded into the same directory as the notebook

In [443]:
if os.path.isfile(ncbi_uncompressed_file_path):
    print(f"{ncbi_uncompressed_file_path} is already present in the current working directory.")
else:
    if os.path.isfile(ncbi_compressed_file_path):
        print(f"{ncbi_compressed_file_path} is already present in the current working directory.")
    else:
        print(f"{ncbi_compressed_file_path} needs to be downloaded")
        ncbi_response = requests.get(ncbi_duckdb_url)
        with open(ncbi_compressed_file_path, "wb") as f:
            f.write(ncbi_response.content)
        # ~ 2 minutes @ 250 Mbps
    
    # Unzip the compressed file and save the extracted file in target directory
    print(f"{ncbi_compressed_file_path} needs to be unpacked")
    with gzip.open(ncbi_compressed_file_path, "rb") as f_in:
        with open(ncbi_uncompressed_file_path, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

    # ~ 2 minutes

ncbi_biosamples.duckdb is already present in the current working directory.


In [444]:
ncbi_conn = duckdb.connect(database=ncbi_uncompressed_file_path, read_only=True)

In [445]:
envo_adapter = get_adapter(envo_adapter_string)

In [446]:
anchor_descendants = get_curie_descendants_label_dict(semantic_anchor, [IS_A], envo_adapter)

In [447]:
anchor_descendants_lod = curie_descendants_label_dict_to_lod(anchor_descendants)

In [448]:
anchor_descendants_frame = curie_descendants_label_lod_to_df(anchor_descendants_lod)

In [449]:
anchor_descendants_frame

,curie,label
0,ENVO:00001999,marine water body
1,ENVO:01000188,tropical savanna biome
2,ENVO:00000487,paternoster lake
3,ENVO:01000860,temperate marine upwelling biome
4,ENVO:01000199,mediterranean forest biome
...,...,...
1731,ENVO:01000429,burrow
1732,ENVO:01000431,mixed forest
1733,ENVO:01000536,factory
1734,ENVO:00000873,freshwater biome


----

In [450]:
sv = get_schemaview_from_source(previous_submission_schema_url)

In [451]:
# todo break out slow steps into its own cell

try:
    CONTEXT_ENUM_def = sv.get_enum(CONTEXT_ENUM)
    context_pvs_keys = list(CONTEXT_ENUM_def.permissible_values.keys())
except AttributeError as e:
    # Handle the AttributeError
    print(f"An AttributeError occurred: {e}")
    context_pvs_keys =[]
    

In [452]:
print(context_pvs_keys)

['active permafrost layer [ENVO:04000009]', 'alas [ENVO:00000438]', 'badland [ENVO:00000127]', 'beach [ENVO:00000091]', 'butte [ENVO:00000287]', 'caldera [ENVO:00000096]', 'campground [ENVO:01000935]', 'canyon [ENVO:00000169]', 'cave [ENVO:00000067]', 'channel [ENVO:03000117]', 'cirque [ENVO:00000155]', 'cliff [ENVO:00000087]', 'crater [ENVO:00000514]', 'crevasse [ENVO:00000320]', 'cryosphere [ENVO:03000143]', 'dam [ENVO:00000074]', 'desert [ENVO:01001357]', 'drainage basin [ENVO:00000291]', 'drumlin [ENVO:00000276]', 'dry lake [ENVO:00000277]', 'dune [ENVO:00000170]', 'endorheic basin [ENVO:00000551]', 'escarpment [ENVO:00000280]', 'esker [ENVO:00000282]', 'farm [ENVO:00000078]', 'fen [ENVO:00000232]', 'fjord [ENVO:00000039]', 'flood plain [ENVO:00000255]', 'frost heave [ENVO:01001568]', 'fumarole [ENVO:00000216]', 'garden [ENVO:00000011]', 'glacier [ENVO:00000133]', 'greenhouse [ENVO:03600087]', 'harbour [ENVO:00000463]', 'hill [ENVO:00000083]', 'hummock [ENVO:00000516]', 'isthmus [E

In [453]:
initially_parsed_context_pvs = parse_hierarchically_underscored_strings(context_pvs_keys)

In [454]:
deduped_context_pvs = dedupe_underscoreless_pvs(initially_parsed_context_pvs)

In [455]:
pv_validation_results = validate_curie_label_list_dict(deduped_context_pvs, envo_adapter, print_flag=True)

In [456]:
pv_validation_results

{'problems': [],
 'valids': [{'curie': 'ENVO:04000009', 'label': 'active permafrost layer'},
  {'curie': 'ENVO:00000438', 'label': 'alas'},
  {'curie': 'ENVO:00000127', 'label': 'badland'},
  {'curie': 'ENVO:00000091', 'label': 'beach'},
  {'curie': 'ENVO:00000287', 'label': 'butte'},
  {'curie': 'ENVO:00000096', 'label': 'caldera'},
  {'curie': 'ENVO:01000935', 'label': 'campground'},
  {'curie': 'ENVO:00000169', 'label': 'canyon'},
  {'curie': 'ENVO:00000067', 'label': 'cave'},
  {'curie': 'ENVO:03000117', 'label': 'channel'},
  {'curie': 'ENVO:00000155', 'label': 'cirque'},
  {'curie': 'ENVO:00000087', 'label': 'cliff'},
  {'curie': 'ENVO:00000514', 'label': 'crater'},
  {'curie': 'ENVO:00000320', 'label': 'crevasse'},
  {'curie': 'ENVO:03000143', 'label': 'cryosphere'},
  {'curie': 'ENVO:00000074', 'label': 'dam'},
  {'curie': 'ENVO:01001357', 'label': 'desert'},
  {'curie': 'ENVO:00000291', 'label': 'drainage basin'},
  {'curie': 'ENVO:00000276', 'label': 'drumlin'},
  {'curie': '

----

In [457]:
if os.path.isfile(all_nmdc_biosamples_file):
    print(f"{all_nmdc_biosamples_file} is present in the current working directory and will be read into all_nmdc_biosamples.")
    # with open(all_nmdc_biosamples_file, 'r') as file:
    #     all_nmdc_biosamples = yaml.full_load(file)
    # read as json
    with open(all_nmdc_biosamples_file, 'r') as f:
        all_nmdc_biosamples = json.load(f)

else:
    print(f"All NMDC Biosamples need to be fetched and saved to {all_nmdc_biosamples_file}")
    all_nmdc_biosamples = get_docs_from_nmdc_collection(NMDC_RUNTIME_BASE_URL,
                                               BIOSAMPLE_SET_COLLECTION)
    # with open(all_nmdc_biosamples_file, 'w') as file:
    #     documents = yaml.dump(all_nmdc_biosamples, file)
    # save as json
    with open(all_nmdc_biosamples_file, 'w') as f:
        json.dump(all_nmdc_biosamples, f)

# this saves network traffic. could use JSON for faster performance. 
# 1 minute for network fetch and JSON write?!
# 1 minute for yaml read
# instantaneous for JSON read?

all_nmdc_biosamples.json is present in the current working directory and will be read into all_nmdc_biosamples.


In [458]:
env_pacakge_overrides = tsv_to_dict_of_dicts(env_package_override_file, 'id')

In [459]:
# todo show env_pacakge_overrides as a data frame
#   with some other columns for context?

In [460]:
biosample_contexts_lod = biosamples_lod_context_extractor(all_nmdc_biosamples, envo_adapter,
                                                          my_env_pacakge_overrides=env_pacakge_overrides)

# ~ 10 seconds, lots of logging

Overriding env_package for biosample nmdc:bsm-11-0k8nkx16 from  to plant-associated
Overriding env_package for biosample nmdc:bsm-11-19v98823 from  to plant-associated
Overriding env_package for biosample nmdc:bsm-11-1yvac190 from  to plant-associated
Overriding env_package for biosample nmdc:bsm-11-28kgw077 from  to plant-associated
Overriding env_package for biosample nmdc:bsm-11-2hswww54 from  to hydrocarbon resources-fluids_swabs
Overriding env_package for biosample nmdc:bsm-11-34przm31 from  to plant-associated
Overriding env_package for biosample nmdc:bsm-11-35m0rm03 from  to hydrocarbon resources-fluids_swabs
Overriding env_package for biosample nmdc:bsm-11-3636w778 from  to plant-associated
Overriding env_package for biosample nmdc:bsm-11-3nffqc45 from  to plant-associated
Overriding env_package for biosample nmdc:bsm-11-3nhng665 from  to plant-associated
Overriding env_package for biosample nmdc:bsm-11-3r4g4610 from  to hydrocarbon resources-fluids_swabs
Overriding env_package

In [461]:
nmdc_biosample_contexts_frame = pd.DataFrame(biosample_contexts_lod)

In [462]:
# print a value count for the normalized_env_package column
print("Value counts for normalized_env_package column:")
print(nmdc_biosample_contexts_frame['normalized_env_package'].value_counts(dropna=False))

Value counts for normalized_env_package column:
normalized_env_package
                                                   5838
soil                                               1707
plant-associated                                    401
water                                               192
miscellaneous natural or artificial environment     140
host-associated                                      61
hydrocarbon resources-fluids_swabs                   23
Name: count, dtype: int64


In [463]:
package_predictions = predict_from_normalized_env_packages(nmdc_biosample_contexts_frame, envo_adapter)

# these predictions often have a f1 of 1.00
# many people might find that hard to believe

(8362, 14)
                                                 precision    recall  f1-score   support

                                host-associated       1.00      1.00      1.00        16
             hydrocarbon resources-fluids_swabs       1.00      1.00      1.00         7
miscellaneous natural or artificial environment       1.00      1.00      1.00        50
                               plant-associated       1.00      1.00      1.00       122
                                           soil       1.00      1.00      1.00       506
                                          water       1.00      1.00      1.00        57

                                       accuracy                           1.00       758
                                      macro avg       1.00      1.00      1.00       758
                                   weighted avg       1.00      1.00      1.00       758



In [464]:
nmdc_biosample_contexts_frame['predicted_env_package'] = package_predictions

In [465]:
nmdc_biosample_contexts_frame.shape

(8362, 15)

In [466]:
nmdc_biosample_contexts_frame = nmdc_biosample_contexts_frame[
    nmdc_biosample_contexts_frame['predicted_env_package'] == nmdc_package_selector]

In [467]:
nmdc_biosample_contexts_frame.shape

(401, 15)

----

In [468]:
ncbi_frame = ncbi_conn.execute(ncbi_query).fetchdf()

In [469]:
ncbi_frame.insert(0, 'serial_number', range(1, len(ncbi_frame) + 1))

In [470]:
# includes values with counts of one... useful for discovering drag-down submissions?

In [471]:
ncbi_frame['content_list'] = ncbi_frame['content'].str.split('|')

In [472]:
## diagnostically useful, but why are we saving this and not all of the intermediate dataframes?
# ncbi_frame.to_csv("ncbi_frame.tsv", sep="\t", index=False)

In [473]:
# todo is there any reason to not do this ?
ncbi_frame = ncbi_frame[ncbi_frame['content'].notna() & (ncbi_frame['content'] != '')]

In [474]:
ncbi_frame['content_count'] = ncbi_frame['content_list'].apply(len)

In [475]:
ncbi_frame.shape

(7696, 5)

In [476]:
ncbi_frame = ncbi_frame.explode('content_list').reset_index(drop=True)

In [477]:
ncbi_frame.shape

(7817, 5)

In [478]:
# how many content_list strings contain envo multiple times now?

In [479]:
ncbi_frame['envo_count'] = ncbi_frame['content_list'].str.lower().str.count("envo")

In [480]:
ncbi_frame['envo_count'].value_counts()

envo_count
0    7210
1     589
3      15
2       2
7       1
Name: count, dtype: int64

doesn't split multiple annotation strings delimited with something other than '|'
annotations with no curies but multiple strings will be "annotated" with OAK, but currently only the one best OAK annotation is kept

In [481]:
ncbi_frame[['extracted_label', 'extracted_curie']] = ncbi_frame['content_list'].apply(parse_curie_label)

In [482]:
parse_failures = ncbi_frame[
    (ncbi_frame['envo_count'] > 0) & (ncbi_frame['extracted_curie'].isna() | (ncbi_frame['extracted_curie'] == ''))]


In [483]:
parse_failures

,serial_number,content,sample_count,content_list,content_count,envo_count,extracted_label,extracted_curie
164,162,Tropical woodland biome [ENVO01000220],223,Tropical woodland biome [ENVO01000220],1,1,Tropical woodland biome [ENVO01000220],None
400,387,Tropical shrubland biome [ENVO01000214],77,Tropical shrubland biome [ENVO01000214],1,1,Tropical shrubland biome [ENVO01000214],None
840,810,ENVO：00000316,19,ENVO：00000316,1,1,ENVO：00000316,None
5710,5609,ENVO,1,ENVO,1,1,ENVO,None


Should we try parsing on additional CURIE delimiters? Or no delimiter?


In [484]:
ncbi_frame['real_label'] = ncbi_frame['extracted_curie'].apply(envo_adapter.label)

In [485]:
# Apply the annotation function to each row in the 'label' column
ncbi_frame['longest_annotation_curie'] = ncbi_frame['extracted_label'].apply(
    lambda x: get_longest_annotation_curie(x, envo_adapter, MIN_ANNOTATION_LEN))

# this cell only takes ~ 1 minute, but generates a lot of "ERRORS" and WARNINGS in a red fornt
#   while loading the ontologies that are used for annotating


ERROR:root:Skipping statements(subject=ENVO:00000112,predicate=oio:hasDbXref,object=<http://www.eionet.europa.eu/gemet/concept/8704>,value=None,datatype=None,language=None,); ValueError: <http://www.eionet.europa.eu/gemet/concept/8704> is not a valid URI or CURIE
ERROR:root:Skipping statements(subject=ENVO:00001996,predicate=oio:hasDbXref,object=<https://en.wikipedia.org/wiki/Acid_mine_drainage>,value=None,datatype=None,language=None,); ValueError: <https://en.wikipedia.org/wiki/Acid_mine_drainage> is not a valid URI or CURIE
ERROR:root:Skipping statements(subject=ENVO:01000225,predicate=oio:hasDbXref,object=<https://www.worldwildlife.org/biomes/tropical-and-subtropical-dry-broadleaf-forests>,value=None,datatype=None,language=None,); ValueError: <https://www.worldwildlife.org/biomes/tropical-and-subtropical-dry-broadleaf-forests> is not a valid URI or CURIE
ERROR:root:Skipping statements(subject=ENVO:01000227,predicate=oio:hasDbXref,object=<https://www.worldwildlife.org/biomes/tropical

In [486]:
ncbi_frame['longest_annotation_label'] = ncbi_frame['longest_annotation_curie'].apply(envo_adapter.label)

In [487]:
ncbi_frame

,serial_number,content,sample_count,content_list,content_count,envo_count,extracted_label,extracted_curie,real_label,longest_annotation_curie,longest_annotation_label
0,1,not applicable,15057,not applicable,1,0,not applicable,None,None,None,None
1,2,NA,12748,NA,1,0,NA,None,None,None,None
2,3,not collected,8348,not collected,1,0,not collected,None,None,None,None
3,4,rhizosphere,6916,rhizosphere,1,0,rhizosphere,None,None,ENVO:00005801,rhizosphere
4,5,woodland,6401,woodland,1,0,woodland,None,None,ENVO:00000057,mangrove swamp
...,...,...,...,...,...,...,...,...,...,...,...
7812,7693,rubber plantation,1,rubber plantation,1,0,rubber plantation,None,None,ENVO:00000164,rubber plantation
7813,7694,Laboratory-2-4,1,Laboratory-2-4,1,0,Laboratory-2-4,None,None,None,None
7814,7695,onion field,1,onion field,1,0,onion field,None,None,ENVO:01000352,field
7815,7696,Rhizosphere of Allisonia spp.,1,Rhizosphere of Allisonia spp.,1,0,Rhizosphere of Allisonia spp.,None,None,ENVO:00005801,rhizosphere


In [488]:
if os.path.isfile(gold_data_file_name):
    print(f"{gold_data_file_name} is already present in the current working directory.")
else:
    print(f"{gold_data_file_name} needs to be downloaded")
    gold_response = requests.get(gold_data_url)
    with open(gold_data_file_name, "wb") as f:
        f.write(gold_response.content)
        # ~ 10 seconds  @ 250 Mbps

goldData.xlsx is already present in the current working directory.


Expect to see

> /home/mark/.cache/pypoetry/virtualenvs/nmdc-submission-schema-DC6HKp4p-py3.10/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")

In [489]:
if os.path.isfile(gold_csv_file_name):
    print(f"{gold_csv_file_name} is present in the current working directory and will be parsed into gold_biosamples_frame.")
    gold_biosamples_frame = pd.read_csv(gold_csv_file_name, sep="\t")
else:
    print(f"gold_biosamples_frame and {gold_csv_file_name} need to be created")
    gold_biosamples_frame = pd.read_excel(gold_data_file_name, sheet_name=BIOSAMPLES_SHEET)
    gold_biosamples_frame.to_csv("gold_biosamples.csv", index=False, sep="\t")
    # 2 minutes

gold_biosamples.csv is present in the current working directory and will be parsed into gold_biosamples_frame.


In [490]:
gold_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'] = gold_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'].fillna(
    0).astype(int)


In [491]:
gold_biosamples_frame

,BIOSAMPLE GOLD ID,BIOSAMPLE NAME,BIOSAMPLE NCBI TAX ID,BIOSAMPLE NCBI TAX NAME,BIOSAMPLE SAMPLE COLLECTION SITE,BIOSAMPLE SAMPLE COLLECTION DATE,BIOSAMPLE GEOGRAPHIC LOCATION,BIOSAMPLE LATITUDE,BIOSAMPLE LONGITUDE,BIOSAMPLE ECOSYSTEM PATH ID,BIOSAMPLE ECOSYSTEM,BIOSAMPLE ECOSYSTEM CATEGORY,BIOSAMPLE ECOSYSTEM TYPE,BIOSAMPLE ECOSYSTEM SUBTYPE,BIOSAMPLE SPECIFIC ECOSYSTEM
0,Gb0011929,"GEBA_MDM Biosample from Great Boiling Spring, ...",749907.0,sediment metagenome,Sediment,NaN,"Great Boiling Spring (GBS), Nevada",40.661433,-119.366250,3992,Environmental,Aquatic,Thermal springs,Hot (42-90C),Unclassified
1,Gb0035601,Compost enrichment cellulose adapted microbial...,702656.0,compost metagenome,single cell,2012-10-31,USA: California: Emeryville: Joint Bio-Energy ...,37.840800,-122.289600,6039,Engineered,Solid waste,Green waste,Composting,Unclassified
2,Gb0035602,Compost enrichment cellulose adapted microbial...,702656.0,compost metagenome,single cell,2012-10-31,USA: California: Emeryville: Joint Bio-Energy ...,37.840800,-122.289600,6039,Engineered,Solid waste,Green waste,Composting,Unclassified
3,Gb0035635,Compost enrichment cellulose adapted microbial...,702656.0,compost metagenome,single cell,2012-10-31,USA: California: Emeryville: Joint Bio-Energy ...,37.840800,-122.289600,6039,Engineered,Solid waste,Green waste,Composting,Unclassified
4,Gb0035638,Compost enrichment cellulose adapted microbial...,702656.0,compost metagenome,single cell,2012-10-31,USA: California: Emeryville: Joint Bio-Energy ...,37.840800,-122.289600,6039,Engineered,Solid waste,Green waste,Composting,Unclassified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211608,Gb0405291,Freshwater biofilm microbial communities from ...,718308.0,biofilm metagenome,creek biofilm,2023-07-26,"USA: Lewis Run NEON Field Site, Briggs, Virginia",39.095630,-77.983216,8389,Environmental,Aquatic,Freshwater,Creek,Biofilm
211609,Gb0405292,Freshwater biofilm microbial communities from ...,718308.0,biofilm metagenome,creek biofilm,2023-07-26,"USA: Lewis Run NEON Field Site, Briggs, Virginia",39.095630,-77.983216,8389,Environmental,Aquatic,Freshwater,Creek,Biofilm
211610,Gb0405293,Freshwater microbial communities from Lake Sug...,449393.0,freshwater metagenome,lake water,2023-08-01,"USA: Lake Suggs NEON Field Site, Melrose, Florida",29.688140,-82.017318,4179,Environmental,Aquatic,Freshwater,Lake,Unclassified
211611,Gb0405294,Freshwater microbial communities from Prairie ...,449393.0,freshwater metagenome,lake water,2023-08-08,"USA: Prairie Lake NEON Field Site, Vashti, Nor...",47.159710,-99.118723,4179,Environmental,Aquatic,Freshwater,Lake,Unclassified


In [492]:
# Determine the filenames and target directory
goldterms_compressed_filename = urlparse(goldterms_semsql_url).path.split('/')[-1]
goldterms_filename = os.path.splitext(goldterms_compressed_filename)[0]
target_dir = os.path.join("..", "..")  # Two levels up

# Print to confirm the filenames
print(goldterms_filename)

goldterms.db


In [493]:
# Fetch the contents from the URL and save compressed file in target directory
goldterms_response = requests.get(goldterms_semsql_url)
goldterms_compressed_file_path = os.path.join(target_dir, goldterms_compressed_filename)
with open(goldterms_compressed_file_path, "wb") as f:
    f.write(goldterms_response.content)
    
# ~ 1 second

In [494]:
# Unzip the compressed file and save the extracted file in target directory
goldterms_uncompressed_file_path = os.path.join(target_dir, goldterms_filename)
with gzip.open(goldterms_compressed_file_path, "rb") as f_in:
    with open(goldterms_uncompressed_file_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

# ~ 1 second

In [495]:
# that's all fast. don't bother caching

In [496]:
goldterms_conn = sqlite3.connect(goldterms_uncompressed_file_path)

In [497]:
goldterms_subjects = pd.read_sql_query(goldterms_subclass_query, goldterms_conn)

In [498]:
goldterms_subjects['path_id'] = goldterms_subjects['subject'].str.extract(r'GOLDTERMS:(\d+)')

In [499]:
goldterms_subjects

,subject,path_id


In [500]:
gold_path_ids = goldterms_subjects['path_id'].dropna().unique().tolist()
gold_path_ids = [int(my_id) for my_id in gold_path_ids]


In [501]:
gold_env_filtered_biosamples_frame = gold_biosamples_frame[
    gold_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'].isin(gold_path_ids)]


In [502]:
gold_env_filtered_biosamples_frame

,BIOSAMPLE GOLD ID,BIOSAMPLE NAME,BIOSAMPLE NCBI TAX ID,BIOSAMPLE NCBI TAX NAME,BIOSAMPLE SAMPLE COLLECTION SITE,BIOSAMPLE SAMPLE COLLECTION DATE,BIOSAMPLE GEOGRAPHIC LOCATION,BIOSAMPLE LATITUDE,BIOSAMPLE LONGITUDE,BIOSAMPLE ECOSYSTEM PATH ID,BIOSAMPLE ECOSYSTEM,BIOSAMPLE ECOSYSTEM CATEGORY,BIOSAMPLE ECOSYSTEM TYPE,BIOSAMPLE ECOSYSTEM SUBTYPE,BIOSAMPLE SPECIFIC ECOSYSTEM


In [503]:
goldterms_context_frame = pd.read_sql_query(goldterms_envo_query, goldterms_conn)

In [504]:
goldterms_context_frame['object_label'] = goldterms_context_frame['object'].apply(envo_adapter.label)

In [505]:
goldterms_context_frame['path_id'] = goldterms_context_frame['subject'].str.extract(r'GOLDTERMS:(\d+)')

In [506]:
goldterms_context_frame

,stanza,subject,predicate,object,value,datatype,language,graph,object_label,path_id
0,GOLDTERMS:Engineered-Bioreactor-Anaerobic-Soft...,GOLDTERMS:Engineered-Bioreactor-Anaerobic-Soft...,mixs:env_local,OBI:0001046,None,None,None,None,None,NaN
1,GOLDTERMS:Engineered-Bioreactor-DHS-reactor,GOLDTERMS:Engineered-Bioreactor-DHS-reactor,mixs:env_local,OBI:0001046,None,None,None,None,None,NaN
2,GOLDTERMS:Engineered-Bioreactor-Membrane-biore...,GOLDTERMS:Engineered-Bioreactor-Membrane-biore...,mixs:env_local,OBI:0001046,None,None,None,None,None,NaN
3,GOLDTERMS:Engineered-Bioreactor-Membrane-biore...,GOLDTERMS:Engineered-Bioreactor-Membrane-biore...,mixs:env_local,OBI:0001046,None,None,None,None,None,NaN
4,GOLDTERMS:Engineered-Bioreactor-Passive-biorea...,GOLDTERMS:Engineered-Bioreactor-Passive-biorea...,mixs:env_local,OBI:0001046,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...
296,GOLDTERMS:5838,GOLDTERMS:5838,mixs:env_local,OBI:0001046,None,None,None,None,None,5838
297,GOLDTERMS:5841,GOLDTERMS:5841,mixs:env_local,ENVO:00000076,None,None,None,None,mine,5841
298,GOLDTERMS:5843,GOLDTERMS:5843,mixs:env_local,OBI:0001046,None,None,None,None,None,5843
299,GOLDTERMS:5846,GOLDTERMS:5846,mixs:env_local,OBI:0001046,None,None,None,None,None,5846


In [507]:
# Fill NaN values in 'BIOSAMPLE ECOSYSTEM PATH ID' with 0 and convert to int
gold_env_filtered_biosamples_frame['BIOSAMPLE ECOSYSTEM PATH ID'] = gold_env_filtered_biosamples_frame[
    'BIOSAMPLE ECOSYSTEM PATH ID'].fillna(0).astype(int)

# Drop rows with NaN in 'path_id' in goldterms_context_frame
goldterms_context_frame = goldterms_context_frame.dropna(subset=['path_id'])

# Convert 'path_id' to int
goldterms_context_frame['path_id'] = goldterms_context_frame['path_id'].astype(int)

# Perform the left merge
gold_env_filtered_biosamples_with_inferred = gold_env_filtered_biosamples_frame.merge(
    goldterms_context_frame,
    left_on='BIOSAMPLE ECOSYSTEM PATH ID',
    right_on='path_id',
    how='left'
)


In [508]:
gold_env_filtered_biosamples_with_inferred

,BIOSAMPLE GOLD ID,BIOSAMPLE NAME,BIOSAMPLE NCBI TAX ID,BIOSAMPLE NCBI TAX NAME,BIOSAMPLE SAMPLE COLLECTION SITE,BIOSAMPLE SAMPLE COLLECTION DATE,BIOSAMPLE GEOGRAPHIC LOCATION,BIOSAMPLE LATITUDE,BIOSAMPLE LONGITUDE,BIOSAMPLE ECOSYSTEM PATH ID,...,stanza,subject,predicate,object,value,datatype,language,graph,object_label,path_id


# GOLDTERMS only approach

In [509]:
goldterms_result = pd.read_sql_query(extension_query, goldterms_conn)

In [510]:
goldterms_result

,subject,predicate,content
0,GOLDTERMS:Host-associated-Plants-Macrophytes,mixs:host_taxon,NCBITaxon:3193
1,GOLDTERMS:Host-associated-Plants-Macrophytes,mixs:mixs_extension,mixs:PlantAssociated
2,GOLDTERMS:Host-associated-Plants-Macrophytes,rdfs:label,Host-associated > Plants > Macrophytes
3,GOLDTERMS:3938,mixs:host_taxon,NCBITaxon:3193
4,GOLDTERMS:3938,mixs:mixs_extension,mixs:PlantAssociated
...,...,...,...
198,GOLDTERMS:5826,mixs:anatomical_site,PO:0009046
199,GOLDTERMS:5826,mixs:env_medium,FOODON:03315553
200,GOLDTERMS:5826,mixs:host_taxon,NCBITaxon:3193
201,GOLDTERMS:5826,mixs:mixs_extension,mixs:PlantAssociated


In [511]:
# # todo: save this kind of content before subsetting on an environment
# #   the subsetting is currently baked into the query
# 
# # see also goldterms_queries.ipynb in MAM's Collab
# goldterms_result.to_csv("goldterms_single_environment_mappings_long.tsv", sep="\t", index=False)

In [512]:
goldterms_only_curies = goldterms_result.loc[goldterms_result['predicate'] == gold_context_selector, 'content']


In [513]:
goldterms_only_curies = goldterms_only_curies.unique().tolist()

In [514]:
goldterms_only_curies

['ENVO:00005801']

----

In [515]:
anchor_curies = list(anchor_descendants_frame['curie'])
legacy_pv_curies = [i['curie'] for i in pv_validation_results['valids']]

biome_curies = list(envo_adapter.descendants(BIOME, predicates=[IS_A])) # 
terrestrial_biome_curies = list(envo_adapter.descendants(TERRESTRIAL_BIOME, predicates=[IS_A]))
aquatic_biome_curies = list(envo_adapter.descendants(AQUATIC_BIOME, predicates=[IS_A]))
abp_curies = list(envo_adapter.descendants(ABP, predicates=[IS_A]))
env_sys_curies = list(envo_adapter.descendants(ENVIRONMENTAL_SYSTEM, predicates=[IS_A]))
env_mat_curies = list(envo_adapter.descendants(ENVIRONMENTAL_MATERIAL, predicates=[IS_A]))
obsoletes_curies = list(envo_adapter.obsoletes())

soil_curies = list(envo_adapter.descendants(SOIL, predicates=[IS_A])) # 
liquid_water_curies = list(envo_adapter.descendants(LIQUID_WATER, predicates=[IS_A])) # 
water_ice_curies = list(envo_adapter.descendants(WATER_ICE, predicates=[IS_A])) # 

human_construction_curies = list(envo_adapter.descendants(HUMAN_CONSTRUCTION, predicates=[IS_A])) #
building_curies = list(envo_adapter.descendants(BUILDING, predicates=[IS_A])) #
building_part_curies = list(envo_adapter.descendants(BUILDING_PART, predicates=[IS_A])) #


In [516]:
include_in_rows = set()

In [517]:
include_in_rows.update(anchor_curies)

In [518]:
include_in_rows.update(legacy_pv_curies)

In [519]:
include_in_rows.update(nmdc_biosample_contexts_frame[nmdc_context_selector])

In [520]:
include_in_rows.update(ncbi_frame['extracted_curie'])

In [521]:
include_in_rows.update(ncbi_frame['longest_annotation_curie'])

In [522]:
include_in_rows.update(gold_env_filtered_biosamples_with_inferred['object'])

In [523]:
include_in_rows.update(goldterms_only_curies)

In [524]:
rows_lod = []

# Begin constructing the voting sheet

In [525]:
for curie in include_in_rows:
    if curie is None:
        continue
        
    # ONCE AGAIN, assuming that EnvO is the only ontology we'll check against
    current_ancestors = list(envo_adapter.ancestors(curie, predicates=[IS_A])) # vs legacy_pv_curies
    ancestors_in_enum_count = len(set(current_ancestors) & set(legacy_pv_curies))
    
    current_descendants  = list(envo_adapter.descendants(curie, predicates=[IS_A])) # vs legacy_pv_curies
    descendants_in_enum_count  = len(set(current_descendants) & set(legacy_pv_curies))
    
    
    row = {
        'curie': curie,
        'label': envo_adapter.label(curie),
        'envo_native': False,
        'obsolete': False,
        comparison_enum_column_name: False,
        'ancestors_in_enum_count': ancestors_in_enum_count,
        'descendants_in_enum_count': descendants_in_enum_count,
        'abp': False,
        'env_sys': False,
        'biome': False,
        'terrestrial_biome': False,
        'aquatic_biome': False,
        'env_mat': False,
        'soil': False,
        'liquid water': False,
        'water ice': False,
        'human_construction': False,
        'building': False,
        'building_part': False,
        'goldterms_mappings': False,
    }
        
    if curie in biome_curies:
        row['biome'] = True
    if curie in terrestrial_biome_curies:
        row['terrestrial_biome'] = True
    if curie in aquatic_biome_curies:
        row['aquatic_biome'] = True
    if curie in abp_curies:
        row['abp'] = True
    if curie in env_sys_curies:
        row['env_sys'] = True
    if curie in env_mat_curies:
        row['env_mat'] = True
    if curie in soil_curies:
        row['soil'] = True
    if curie in liquid_water_curies:
        row['liquid water'] = True
    if curie in water_ice_curies:
        row['water ice'] = True
    if curie in human_construction_curies:
        row['human_construction'] = True
    if curie in building_curies:
        row['building'] = True
    if curie in building_part_curies:
        row['building_part'] = True
    if curie in legacy_pv_curies:
        row[comparison_enum_column_name] = True
    if curie in obsoletes_curies:
        row['obsolete'] = True
    if curie in goldterms_only_curies:
        row['goldterms_mappings'] = True
        
    try:
        prefix, local_id = curie.split(':')
        if prefix and prefix == 'ENVO' and row['label'] is not None:
            row['envo_native'] = True
    except Exception as e:
        # Print the exception message
        print(f"An error occurred: {e} trying to split {curie}")

    rows_lod.append(row)

# 2 minutes


# ^ Construction of the voting sheet

In [526]:
rows_frame = pd.DataFrame(rows_lod)

In [527]:
rows_frame

,curie,label,envo_native,obsolete,EnvLocalScaleSoilEnum_11_1,ancestors_in_enum_count,descendants_in_enum_count,abp,env_sys,biome,terrestrial_biome,aquatic_biome,env_mat,soil,liquid water,water ice,human_construction,building,building_part,goldterms_mappings
0,ENVO:01001107,rockslide lake,True,False,False,0,0,True,False,False,False,False,False,False,False,False,False,False,False,False
1,ENVO:00000061,underground water body,True,False,False,0,0,True,False,False,False,False,False,False,False,False,False,False,False,False
2,ENVO:00009726,strand crack,True,False,False,0,0,True,False,False,False,False,False,False,False,False,False,False,False,False
3,ENVO:00000537,boulder field,True,False,False,0,0,True,False,False,False,False,False,False,False,False,False,False,False,False
4,ENVO:00000401,spatter cone,True,False,False,0,0,True,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,ENVO:01000037,oceanic bathypelagic zone biome,True,False,False,0,0,True,True,True,False,True,False,False,False,False,False,False,False,False
2074,ENVO:03501341,community health centre,True,False,False,0,0,True,False,False,False,False,False,False,False,False,True,False,False,False
2075,ENVO:01001526,frozen soil,True,False,False,0,0,True,False,False,False,False,True,True,False,False,False,False,False,False
2076,ENVO:01000410,marine eulittoral zone,True,False,False,0,0,True,False,False,False,False,False,False,False,False,False,False,False,False


In [528]:
nmdc_biosample_scoped_counts = nmdc_biosample_contexts_frame[nmdc_context_selector].value_counts().reset_index()
nmdc_biosample_scoped_counts.columns = ['curie', 'nmdc_scoped_count']


In [529]:
nmdc_biosample_scoped_counts

,curie,nmdc_scoped_count
0,ENVO:01001057,199
1,ENVO:01001442,192
2,PO:0025025,10


In [530]:
# Perform the left merge
rows_frame = rows_frame.merge(
    nmdc_biosample_scoped_counts,
    left_on='curie',
    right_on='curie',
    how='left'
)

In [531]:
gold_env_filtered_biosamples_with_inferred

,BIOSAMPLE GOLD ID,BIOSAMPLE NAME,BIOSAMPLE NCBI TAX ID,BIOSAMPLE NCBI TAX NAME,BIOSAMPLE SAMPLE COLLECTION SITE,BIOSAMPLE SAMPLE COLLECTION DATE,BIOSAMPLE GEOGRAPHIC LOCATION,BIOSAMPLE LATITUDE,BIOSAMPLE LONGITUDE,BIOSAMPLE ECOSYSTEM PATH ID,...,stanza,subject,predicate,object,value,datatype,language,graph,object_label,path_id


In [532]:
gold_biosample_scoped_counts = gold_env_filtered_biosamples_with_inferred['object'].value_counts().reset_index()
gold_biosample_scoped_counts.columns = ['curie', 'gold_hybrid_count']

In [533]:
gold_biosample_scoped_counts

,curie,gold_hybrid_count


In [534]:
# Perform the left merge
rows_frame = rows_frame.merge(
    gold_biosample_scoped_counts,
    left_on='curie',
    right_on='curie',
    how='left'
)

In [535]:
rows_frame

,curie,label,envo_native,obsolete,EnvLocalScaleSoilEnum_11_1,ancestors_in_enum_count,descendants_in_enum_count,abp,env_sys,biome,...,env_mat,soil,liquid water,water ice,human_construction,building,building_part,goldterms_mappings,nmdc_scoped_count,gold_hybrid_count
0,ENVO:01001107,rockslide lake,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,False,NaN,NaN
1,ENVO:00000061,underground water body,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,False,NaN,NaN
2,ENVO:00009726,strand crack,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,False,NaN,NaN
3,ENVO:00000537,boulder field,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,False,NaN,NaN
4,ENVO:00000401,spatter cone,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,ENVO:01000037,oceanic bathypelagic zone biome,True,False,False,0,0,True,True,True,...,False,False,False,False,False,False,False,False,NaN,NaN
2074,ENVO:03501341,community health centre,True,False,False,0,0,True,False,False,...,False,False,False,False,True,False,False,False,NaN,NaN
2075,ENVO:01001526,frozen soil,True,False,False,0,0,True,False,False,...,True,True,False,False,False,False,False,False,NaN,NaN
2076,ENVO:01000410,marine eulittoral zone,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,False,NaN,NaN


In [536]:
# 990 rows in https://docs.google.com/spreadsheets/d/12WH3eduBq2qSTy9zVF3n7fyajn6ssLZL/edit?gid=546570706#gid=546570706

In [537]:
# gold and ncbi counts are slightly trickier
# for gold: including mappings only, mappings in hybrid with biosample counts. 
#    Switch to direct biosample counts of GOLD "envo" annotations?
# ncbi: we have extracted curies and annotated curies

In [538]:
# todo move this stuff up to immediately after the creation of ncbi_frame ?

# todo don't accept extracted curie if no real label?
# any kind of string similarity checking for label of annotated curie vs extracted label ?
# look for long runs of curies?
# can we measure the beneficial impact of any of this? current crux: how to distribute counts

ncbi_frame['curie_list'] = ncbi_frame.apply(
    lambda my_row: list({my_row['extracted_curie'], my_row['longest_annotation_curie']} - {None}),
    axis=1
)

ncbi_frame['unique_curie_count'] = ncbi_frame['curie_list'].apply(len)

In [539]:
ncbi_frame

,serial_number,content,sample_count,content_list,content_count,envo_count,extracted_label,extracted_curie,real_label,longest_annotation_curie,longest_annotation_label,curie_list,unique_curie_count
0,1,not applicable,15057,not applicable,1,0,not applicable,None,None,None,None,[],0
1,2,NA,12748,NA,1,0,NA,None,None,None,None,[],0
2,3,not collected,8348,not collected,1,0,not collected,None,None,None,None,[],0
3,4,rhizosphere,6916,rhizosphere,1,0,rhizosphere,None,None,ENVO:00005801,rhizosphere,[ENVO:00005801],1
4,5,woodland,6401,woodland,1,0,woodland,None,None,ENVO:00000057,mangrove swamp,[ENVO:00000057],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7812,7693,rubber plantation,1,rubber plantation,1,0,rubber plantation,None,None,ENVO:00000164,rubber plantation,[ENVO:00000164],1
7813,7694,Laboratory-2-4,1,Laboratory-2-4,1,0,Laboratory-2-4,None,None,None,None,[],0
7814,7695,onion field,1,onion field,1,0,onion field,None,None,ENVO:01000352,field,[ENVO:01000352],1
7815,7696,Rhizosphere of Allisonia spp.,1,Rhizosphere of Allisonia spp.,1,0,Rhizosphere of Allisonia spp.,None,None,ENVO:00005801,rhizosphere,[ENVO:00005801],1


In [540]:
ncbi_frame['unique_curie_count'].value_counts()

unique_curie_count
0    4663
1    3038
2     116
Name: count, dtype: int64

In [541]:
double_curie_frame = ncbi_frame[ncbi_frame['unique_curie_count'] > 1]

In [542]:
double_curie_frame = double_curie_frame[['extracted_curie', 'longest_annotation_curie']]

In [543]:
double_curie_frame = double_curie_frame.drop_duplicates()

In [544]:
double_curie_frame[['extracted_prefix', 'extracted_local_id']] = double_curie_frame['extracted_curie'].str.split(':', expand=True)

In [545]:
double_curie_frame['extracted_local_id_int'] = pd.to_numeric(double_curie_frame['extracted_local_id'], errors='coerce').astype('Int64')

In [546]:
# Ensure extracted_local_id_int is unique and sorted
unique_sorted_series = double_curie_frame['extracted_local_id_int'].dropna().drop_duplicates().sort_values()


In [547]:
# Find stretches
stretches_dict = find_consecutive_stretches_dict(unique_sorted_series)

# pprint.pprint(stretches_dict)

In [548]:
# Convert the stretches dictionary into a DataFrame
stretches_df = stretches_dict_to_long_dataframe(stretches_dict)

In [549]:
stretches_df

,stretch_id,value
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
65,4,1001459
66,4,1001460
67,4,1001461
68,4,1001462


In [550]:
# Perform the left merge
double_curie_frame = double_curie_frame.merge(
    stretches_df,
    left_on='extracted_local_id_int',
    right_on='value',
    how='left'
)

In [551]:
stretch_summary_df = summarize_stretch_groups(double_curie_frame)


In [552]:
stretch_summary_df

,stretch_id,most_common_longest_annotation_curie,fraction
0,1.0,ENVO:00001998,0.500000
1,2.0,ENVO:00000116,1.000000
2,3.0,ENVO:00003081,1.000000
3,4.0,ENVO:01001430,0.942857


In [553]:
# Perform the left merge
double_curie_frame = double_curie_frame.merge(
    stretch_summary_df,
    left_on='stretch_id',
    right_on='stretch_id',
    how='left'
)

In [554]:
drag_evidence_frame = double_curie_frame[double_curie_frame['stretch_id'] >= 1]
drag_evidence_frame = drag_evidence_frame[['extracted_curie', 'longest_annotation_curie']]
drag_evidence_frame['drag_evidence'] = True

In [555]:
drag_evidence_frame

,extracted_curie,longest_annotation_curie,drag_evidence
3,ENVO:01001430,ENVO:00000077,True
16,ENVO:01001442,ENVO:01001246,True
19,ENVO:00000002,ENVO:00001998,True
20,ENVO:00000004,ENVO:00001998,True
21,ENVO:00000005,ENVO:00001998,True
...,...,...,...
105,ENVO:01001456,ENVO:01001430,True
106,ENVO:01001459,ENVO:01001430,True
107,ENVO:01001460,ENVO:01001430,True
111,ENVO:01001462,ENVO:01001430,True


In [556]:
ncbi_frame = ncbi_frame.merge(
    drag_evidence_frame,
    left_on=['extracted_curie', 'longest_annotation_curie'],
    right_on=['extracted_curie', 'longest_annotation_curie'],
    how='left'
)

In [557]:
# Initialize dragless_curie_list with curie_list values
ncbi_frame["dragless_curie_list"] = ncbi_frame["curie_list"]

# Update dragless_curie_list based on the condition
for index, row in ncbi_frame.iterrows():
    if row["drag_evidence"] is True:
        if row["longest_annotation_curie"] is not None:
            ncbi_frame.at[index, "dragless_curie_list"] = [row["longest_annotation_curie"]]
        else:
            ncbi_frame.at[index, "dragless_curie_list"] = []

ncbi_frame['dragless_curie_count'] = ncbi_frame['dragless_curie_list'].apply(len)

In [558]:
ncbi_frame['unique_curie_count'].value_counts()

unique_curie_count
0    4663
1    3038
2     116
Name: count, dtype: int64

In [559]:
ncbi_frame['dragless_curie_count'].value_counts()

dragless_curie_count
0    4663
1    3116
2      38
Name: count, dtype: int64

In [560]:
ncbi_frame.shape

(7817, 16)

In [561]:
ncbi_frame_undisputed = ncbi_frame[ncbi_frame['dragless_curie_count'] <= 1]

In [562]:
ncbi_frame_undisputed.shape

(7779, 16)

In [563]:
ncbi_frame_disputed = ncbi_frame[ncbi_frame['dragless_curie_count'] > 1]

In [564]:
ncbi_frame_disputed.shape

(38, 16)

In [565]:
ncbi_frame_disputed = ncbi_frame_disputed.explode("dragless_curie_list", ignore_index=True)


In [566]:
ncbi_frame_disputed.shape

(76, 16)

In [567]:
ncbi_frame_disputed["dragless_curie_list"] = ncbi_frame_disputed["dragless_curie_list"].apply(lambda x: [x])

In [568]:
# Combine the rows of ncbi_frame_undisputed and ncbi_frame_disputed into a new DataFrame
ncbi_disputes_exploded_frame = pd.concat([ncbi_frame_undisputed, ncbi_frame_disputed], ignore_index=True)


In [569]:
ncbi_disputes_exploded_frame.shape

(7855, 16)

In [570]:
ncbi_disputes_exploded_frame

,serial_number,content,sample_count,content_list,content_count,envo_count,extracted_label,extracted_curie,real_label,longest_annotation_curie,longest_annotation_label,curie_list,unique_curie_count,drag_evidence,dragless_curie_list,dragless_curie_count
0,1,not applicable,15057,not applicable,1,0,not applicable,None,None,None,None,[],0,NaN,[],0
1,2,NA,12748,NA,1,0,NA,None,None,None,None,[],0,NaN,[],0
2,3,not collected,8348,not collected,1,0,not collected,None,None,None,None,[],0,NaN,[],0
3,4,rhizosphere,6916,rhizosphere,1,0,rhizosphere,None,None,ENVO:00005801,rhizosphere,[ENVO:00005801],1,NaN,[ENVO:00005801],1
4,5,woodland,6401,woodland,1,0,woodland,None,None,ENVO:00000057,mangrove swamp,[ENVO:00000057],1,NaN,[ENVO:00000057],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7850,7445,vineyard ENVO:00000301,1,vineyard ENVO:00000301,1,1,vineyard,ENVO:00000301,chaparral area,ENVO:00000116,vineyard,"[ENVO:00000116, ENVO:00000301]",2,NaN,[ENVO:00000301],2
7851,7446,vineyard ENVO:00000208,1,vineyard ENVO:00000208,1,1,vineyard,ENVO:00000208,marine pelagic zone,ENVO:00000116,vineyard,"[ENVO:00000116, ENVO:00000208]",2,NaN,[ENVO:00000116],2
7852,7446,vineyard ENVO:00000208,1,vineyard ENVO:00000208,1,1,vineyard,ENVO:00000208,marine pelagic zone,ENVO:00000116,vineyard,"[ENVO:00000116, ENVO:00000208]",2,NaN,[ENVO:00000208],2
7853,7448,Area of crop land ENVO_01000892,1,Area of crop land ENVO_01000892,1,1,Area of crop land,ENVO:01000892,area of cropland,ENVO:01000635,planetary landmass,"[ENVO:01000892, ENVO:01000635]",2,NaN,[ENVO:01000892],2


In [571]:
ncbi_disputes_exploded_frame['post_explode_curie_count'] = ncbi_disputes_exploded_frame['dragless_curie_list'].apply(len)

In [572]:
ncbi_disputes_exploded_frame['post_explode_curie_count'].value_counts()

post_explode_curie_count
0    4663
1    3192
Name: count, dtype: int64

In [573]:
# Set 'post_explode_curie' to the 0th item in 'dragless_curie_list'
ncbi_disputes_exploded_frame["post_explode_curie"] = ncbi_disputes_exploded_frame["dragless_curie_list"].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None
)

In [574]:

ncbi_biosample_scoped_counts = (
    ncbi_disputes_exploded_frame.groupby("post_explode_curie")["sample_count"].sum().reset_index()
)

ncbi_biosample_scoped_counts.columns = ['curie', 'ncbi_scoped_count']

In [575]:
ncbi_biosample_scoped_counts

,curie,ncbi_scoped_count
0,BFO:0000015,18
1,BFO:0000029,264
2,CHEBI:15377,194
3,CHEBI:22695,38
4,CHEBI:24632,2
...,...,...
592,UBERON:0001913,1
593,UBERON:0001988,3
594,UBERON:0002100,27
595,UBERON:0002416,1473


In [576]:
# Perform the left merge
rows_frame = rows_frame.merge(
    ncbi_biosample_scoped_counts,
    left_on='curie',
    right_on='curie',
    how='left'
)

In [577]:
rows_frame

,curie,label,envo_native,obsolete,EnvLocalScaleSoilEnum_11_1,ancestors_in_enum_count,descendants_in_enum_count,abp,env_sys,biome,...,soil,liquid water,water ice,human_construction,building,building_part,goldterms_mappings,nmdc_scoped_count,gold_hybrid_count,ncbi_scoped_count
0,ENVO:01001107,rockslide lake,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
1,ENVO:00000061,underground water body,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
2,ENVO:00009726,strand crack,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
3,ENVO:00000537,boulder field,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
4,ENVO:00000401,spatter cone,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,ENVO:01000037,oceanic bathypelagic zone biome,True,False,False,0,0,True,True,True,...,False,False,False,False,False,False,False,NaN,NaN,NaN
2074,ENVO:03501341,community health centre,True,False,False,0,0,True,False,False,...,False,False,False,True,False,False,False,NaN,NaN,NaN
2075,ENVO:01001526,frozen soil,True,False,False,0,0,True,False,False,...,True,False,False,False,False,False,False,NaN,NaN,NaN
2076,ENVO:01000410,marine eulittoral zone,True,False,False,0,0,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,187.0


In [578]:
rows_frame.to_csv(output_file_name, sep="\t", index=False)